<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#N-Risky-Assets-Portfolio" data-toc-modified-id="N-Risky-Assets-Portfolio-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>N Risky Assets Portfolio</a></span><ul class="toc-item"><li><span><a href="#Question-1" data-toc-modified-id="Question-1-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Question 1</a></span></li></ul></li></ul></div>

In [1]:
import numpy as np
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats, optimize
import seaborn as sns

config = {
    'figure.facecolor':'w',
    'axes.grid':True,
    'axes.spines.top':False,
    'axes.spines.bottom':False,
    'axes.spines.left':False,
    'axes.spines.right':False,
    'grid.linewidth':0.5,
    'grid.linestyle':'--',
    'figure.constrained_layout.use':True
}

plt.rcParams.update(config)

# N Risky Assets Portfolio

Here's our data:

Given three assets characterized by:

| Asset | Expected Return |
|-------|-----------------|
|   1   |       11        |
|   2   |        9        |
|   3   |       42        |

The Variance-covariance Matrix is defined as:

$$\Omega = \begin{bmatrix}
0.0064 & 0.00336 & 0.011088 \\
0.00336 & 0.0049  & 0.008316 \\
0.011088 & 0.008316 & 0.1089 \\
\end{bmatrix} $$

There is no Risk-free asset.

In [2]:
r = np.array([0.11, 0.09, 0.42])

Ω = np.array([
    [0.0064, 0.00336, 0.011088],
    [0.00336, 0.0049, 0.008316],
    [0.011088, 0.008316, 0.1089]
])

## Question 1

Write down the objective function.

We're still doing what we've always done: we seek weights $w$ to minimize the variance of our portfolio, and the weights should all add up to 1. 

We're going to add one new constraint, that the resulting portfolio have exactly returns $\mu_p$. That is, we want $w^T \mu = \mu_p$. Putting all these requiremnts into a constrained optimization gives:


$$\min_{w}\frac{1}{2}w^T \Sigma w, \quad s.t. \begin{cases} w^T \textbf 1 = 1 \\ w^T \mu = \mu_p \end{cases}$$

**Important note**
You should remember that $y^T X y$, where $y$ is a vector and $X$ is a matrix, is a matrix quadratic equation. It's the linear algebra equivalent to $xy^2$. 


Since this is a constrained optimization, set up a Lagrangian:

$$\mathcal L = \frac{1}{2}w^T \Sigma w - \lambda_1(w^T \mu - \mu_p) - \lambda_2(w^T \textbf 1 - 1)$$

Get FoC:

$$\begin{align}\frac{\partial \mathcal L}{\partial w} = 0 \implies & 2 \Sigma w - \lambda_1 \mu - \lambda_2 \textbf 1 = 0 \\
                \frac{\partial \mathcal L}{\partial \lambda_1} = 0 \implies & w^T \mu - \mu_p = 0 \\
                \frac{\partial \mathcal L}{\partial \lambda_2} = 0 \implies & w^T \textbf 1 - 1 = 0 
\end{align}$$

From here we have a choice. Either:

1. **We do a lot of algebra by hand to get $w$.**

To do this, solve $w = \frac{1}{2}\Sigma^{-1} \lambda_1 \mu + \frac{1}{2}\Sigma^{-1} \lambda_2 \textbf 1$ from the first equation.

We can make this equation a bit prettier if we define $\lambda = \begin{bmatrix} \lambda_1 \\ \lambda_2  \end{bmatrix}$ (shape is $2 \times 1$) and $M = \begin{bmatrix}\mu & \textbf 1 \end{bmatrix}$ (shape is $n \times 2$). Then we get:

$$w = \frac{1}{2}\Sigma^{-1}M\lambda$$

Next we substitute $w$ into the first constraint:

$$\begin{align}w^T \mu &= \mu_p \\
               \mu^T w &= \mu_p \\
               \frac{1}{2}\mu^T \Sigma^{-1} \lambda_1 \mu + \frac{1}{2} \mu^T \Sigma^{-1} \lambda_2 \textbf 1 &= \mu_p \\
                \frac{1}{2} \lambda_1 \mu^T \Sigma^{-1} \mu + \frac{1}{2} \lambda_2 \mu^T \Sigma^{-1} \textbf 1 &= \mu_p
\end{align}$$

And the second:

$$\begin{align} w^T \textbf 1 &= 1 \\
                \textbf 1^T w &= 1 \\
                \frac{1}{2} \lambda_1 \textbf 1^T \Sigma^{-1} \mu + \frac{1}{2} \lambda_2 \textbf 1^T \Sigma^{-1} \textbf 1 &= 1
\end{align}$$

We can stack these two equations into a single matrix equation:

$$\frac{1}{2} \begin{bmatrix} \mu^T \Sigma^{-1} \mu & \mu^T \Sigma^{-1} \textbf 1 \\
                   \textbf 1^T \Sigma^{-1} \mu & \textbf 1^T \Sigma^{-1} \textbf 1 \end{bmatrix} \begin{bmatrix} \lambda_1 \\ \lambda_2 \end{bmatrix} =
                   \begin{bmatrix} \mu_p \\ 1 \end{bmatrix}$$
                   
If you stare at this matrix for a bit:

$$
 \begin{bmatrix} \mu^T \Sigma^{-1} \mu & \mu^T \Sigma^{-1} \textbf 1 \\
                   \textbf 1^T \Sigma^{-1} \mu & \textbf 1^T \Sigma^{-1} \textbf 1 \end{bmatrix} \begin{bmatrix} \lambda_1 \\ \lambda_2 \end{bmatrix}
$$

You will see that it is actually $M^T \Sigma^{-1} M$, with $M$ defined as above. So we have found:

$$\frac{1}{2} B \lambda = \bar{\mu_p}$$

with $\bar{\mu_p} = \begin{bmatrix} \mu_p \\ 1 \end{bmatrix}$ and $B = M^T \Sigma^{-1} M$.

So we can solve this for $\lambda$:
$$\lambda = -2B^{-1} \bar{\mu_p}$$

Then plug this back into the weights:

$$w = \Sigma^{-1}MB^{-1} \bar{\mu_p}$$

2. ** We stack the FoCs into a matrix directly and solve it with a computer**

We take the three FoCs:

$$\begin{align}
2 \Sigma w - \lambda_1 \mu - \lambda_2 \textbf 1 &= 0 \\
w^T \mu - \mu_p &= 0 \\
 w^T \textbf 1 - 1 &= 0 \end{align}$$
 
And re-write the system as a single matrix equation:

$$\begin{bmatrix} 2\Sigma & \mu & \textbf 1 \\
                  \textbf 1^T & 0 & 0 \\
                  \mu^T & 0 & 0 \end{bmatrix} \begin{bmatrix} w \\ \lambda_1 \\ \lambda_2 \end{bmatrix} = \begin{bmatrix} 0 \\ 1 \\ \mu_p \end{bmatrix}$$

Or, in a more compact notation:

$$ A x = b$$

With solution $ x = A^{-1}b$


In [86]:
n = 3
A = np.r_[np.c_[2 * Ω, r, np.ones(n)],
          np.c_[np.ones((1, n)), 0, 0],
          np.c_[r[None], 0, 0]]
A_inv = np.linalg.inv(A)

In [87]:
A

array([[0.0128  , 0.00672 , 0.022176, 0.11    , 1.      ],
       [0.00672 , 0.0098  , 0.016632, 0.09    , 1.      ],
       [0.022176, 0.016632, 0.2178  , 0.42    , 1.      ],
       [1.      , 1.      , 1.      , 0.      , 0.      ],
       [0.11    , 0.09    , 0.42    , 0.      , 0.      ]])

In [88]:
r_p = 0.13
b = np.r_[np.zeros(n), 1, r_p]
sol = (A_inv @ b)
w, lambdas = sol[:-2], sol[-2:]

In [89]:
w

array([0.43907397, 0.46632445, 0.09460158])

In [90]:
w.T @ Ω @ w

0.006304746654329381